In [16]:
import pandas as pd 
import numpy as np
import sklearn as skl
import datasets
from sklearn.model_selection import train_test_split
import os

## Create Train / Validate / Test Set

In [35]:
# 1 is anti vaccine, 2 is pro vaccine, 0 is noise
train_anti_label = {
    'CovidIsACult' : 1,
    'covidskepticscanada': 1,
    'FUCKCHINAVIRUS' : 1,
    'ParentingWithoutFear': 1,
    'TrueAntiVaccination': 1,
    'vaccinesideeffects' : 1,
    'vacci_nation' : 1,
    'NoNewNormal': 1,
}
train_pro_label = {
    'CoronaVaccines' : 2,
    'CovidVaccinated' : 2,
    'VACCINES': 2,
    'YouWillSurviveIt' : 2,
}

noise_label = {
    'aww': 0,
    'KidsAreFuckingStupid': 0,
    'natureismetal': 0,
}


### Submissions

In [41]:
df = pd.DataFrame()
for key in train_anti_label:
    temp = pd.read_pickle(os.path.join('../../../Files/Submissions/train/'+key+'.pickle'))
    print(key, len(temp))
    df = pd.concat([df, temp])

df['label'] = df.subreddit.map(train_anti_label)

CovidIsACult 115
covidskepticscanada 3062
FUCKCHINAVIRUS 171
ParentingWithoutFear 216
TrueAntiVaccination 4394
vaccinesideeffects 467
vacci_nation 1899
NoNewNormal 109292


In [42]:
df2 = pd.DataFrame()
for key in train_pro_label:
    temp = pd.read_pickle(os.path.join('../../../Files/Submissions/train/'+key+'.pickle'))
    print(key, len(temp))
    df2 = pd.concat([df2, temp])

df2['label'] = df2.subreddit.map(train_pro_label)

CoronaVaccines 307
CovidVaccinated 26721
VACCINES 4236
YouWillSurviveIt 725


In [43]:
df3 = pd.DataFrame()
for key in noise_label:
    temp = pd.read_pickle(os.path.join('../../../Files/Submissions/train/'+key+'.pickle'))
    print(key, len(temp))
    df3 = pd.concat([df3, temp])

df3['label'] = df3.subreddit.map(noise_label)

aww 1251041
KidsAreFuckingStupid 52807
natureismetal 37221


In [44]:
print(len(df), len(df2), len(df3))

119616 31989 1341069


In [46]:

anti_train = df.sample(n=23000, random_state=42)
pro_train = df2.sample(n=23000, random_state=42)
noise_train = df3.sample(n=23000 , random_state = 42)

In [47]:
anti_temp = pd.concat([df, anti_train, anti_train]).drop_duplicates(keep=False)
pro_temp = pd.concat([df2, pro_train, pro_train]).drop_duplicates(keep=False)
noise_temp = pd.concat([df3, noise_train, noise_train]).drop_duplicates(keep=False)

In [50]:
train = pd.concat([anti_train, pro_train, noise_train]).sample(frac=1).reset_index(drop=True) #merge and shuffle

In [59]:
anti_val = anti_temp.sample(n=3200, random_state=42)
pro_val = pro_temp.sample(n=6000, random_state=42)
noise_val = noise_temp.sample(n=6000, random_state=42)

In [60]:
valid = pd.concat([anti_val, pro_val, noise_val]).sample(frac=1).reset_index(drop=True)

In [61]:
anti_test = pd.concat([anti_temp, anti_val, anti_val]).drop_duplicates(keep=False)
pro_test = pd.concat([pro_temp, pro_val, pro_val]).drop_duplicates(keep=False)
noise_test = pd.concat([noise_temp, noise_val, noise_val]).drop_duplicates(keep=False)

In [62]:
test = pd.concat([anti_test, pro_test, noise_test]).sample(frac=1).reset_index(drop=True)

In [63]:
print(len(train), len(valid), len(test))

69000 15200 1408474


In [56]:
train.drop('selftext', inplace=True, axis=1)
valid.drop('selftext', inplace=True, axis=1)
test.drop('selftext', inplace=True, axis=1)


In [66]:
train.groupby(by='label').count()

,author,created_utc,id,num_comments,permalink,score,subreddit,title,url
label,,,,,,,,,
0,23000,23000,23000,23000,23000,23000,23000,23000,23000
1,23000,23000,23000,23000,23000,23000,23000,23000,23000
2,23000,23000,23000,23000,23000,23000,23000,23000,23000


In [65]:
valid.groupby(by='label').count()

,author,created_utc,id,num_comments,permalink,score,selftext,subreddit,title,url
label,,,,,,,,,,
0,6000,6000,6000,6000,6000,6000,6000,6000,6000,6000
1,3200,3200,3200,3200,3200,3200,3193,3200,3200,3200
2,6000,6000,6000,6000,6000,6000,5975,6000,6000,6000


In [64]:
test.groupby(by='label').count()

,author,created_utc,id,num_comments,permalink,score,selftext,subreddit,title,url
label,,,,,,,,,,
0,1312069,1312069,1312069,1312069,1312069,1312069,1312068,1312069,1312069,1312069
1,93416,93416,93416,93416,93416,93416,93255,93416,93416,93416
2,2989,2989,2989,2989,2989,2989,2974,2989,2989,2989


In [67]:
train['subreddit'].value_counts()

aww                     21529
NoNewNormal             20957
CovidVaccinated         19255
VACCINES                 3004
TrueAntiVaccination       872
KidsAreFuckingStupid      872
covidskepticscanada       630
natureismetal             599
YouWillSurviveIt          522
vacci_nation              366
CoronaVaccines            219
vaccinesideeffects         85
FUCKCHINAVIRUS             38
ParentingWithoutFear       36
CovidIsACult               16
Name: subreddit, dtype: int64

In [68]:
train.to_pickle('../../../Files/Submissions/train/train_split_submission.pickle')
valid.to_pickle('../../../Files/Submissions/train/val_split_submission.pickle')
test.to_pickle('../../../Files/Submissions/train/test_split_submission.pickle')